In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-17 05:40:26--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.47MB/s    in 0.6s    

2021-11-17 05:40:27 (1.47 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Shoes_v1_00.tsv.gz"), inferSchema=True, sep=r'\t', timestampFormat="yyyy/mm/dd")
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [5]:
print(df.count())
df= df.dropna()

4366916


In [6]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
#Transform for review id table
from pyspark.sql.functions import *
review_id_table = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date(col("review_date"), "yyyy-MM-dd")])
review_id_table.show(5)

+--------------+-----------+----------+--------------+--------------------------------+
|     review_id|customer_id|product_id|product_parent|to_date(review_date, yyyy-MM-dd)|
+--------------+-----------+----------+--------------+--------------------------------+
|R3P2HIOQCIN5ZU|   18069663|B000XB31C0|     265024781|                      2015-08-31|
|R12VVR0WH5Q24V|   16251825|B00CFYZH5W|     259035853|                      2015-08-31|
| RNCCKB6TV5EEF|   20381037|B00S8JNN3Q|     666066660|                      2015-08-31|
|R2NZXYIVCGB13W|     108364|B00XFBPOQG|     448483263|                      2015-08-31|
|R2EQ1TG9IT3OEQ|   45449350|B00SW64Y9W|       7853171|                      2015-08-31|
+--------------+-----------+----------+--------------+--------------------------------+
only showing top 5 rows



In [8]:
review_id_table.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- to_date(review_date, yyyy-MM-dd): date (nullable = true)



In [9]:
review_id_table = review_id_table.withColumnRenamed("to_date(review_date, yyyy-MM-dd)", "review_date")
review_id_table.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R3P2HIOQCIN5ZU|   18069663|B000XB31C0|     265024781| 2015-08-31|
|R12VVR0WH5Q24V|   16251825|B00CFYZH5W|     259035853| 2015-08-31|
| RNCCKB6TV5EEF|   20381037|B00S8JNN3Q|     666066660| 2015-08-31|
|R2NZXYIVCGB13W|     108364|B00XFBPOQG|     448483263| 2015-08-31|
|R2EQ1TG9IT3OEQ|   45449350|B00SW64Y9W|       7853171| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [10]:
product_table = df.dropna()
product_table = df.select("product_id", "product_title").distinct()
product_table = product_table.dropDuplicates()
product_table.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CJL5ES|L.B. Evans Men's ...|
|B002MCVHUQ|New Balance Insol...|
|B00UXBFW6S|Kenox Sling Backp...|
|B002CMM4M6|Capezio Women's 1...|
|B00HNO32BU|Timberland PRO Me...|
+----------+--------------------+
only showing top 5 rows



In [11]:
print(product_table.count())

1900895


In [12]:
customer_table = df.select(["customer_id","star_rating"])
customer_table = df.groupby("customer_id").agg({"star_rating": "count"})
customer_table.show(5)

+-----------+------------------+
|customer_id|count(star_rating)|
+-----------+------------------+
|   16121565|                 5|
|   48146680|                 1|
|   11062912|                 1|
|   51451778|                 1|
|   27920838|                 1|
+-----------+------------------+
only showing top 5 rows



In [13]:
customer_table = customer_table.withColumnRenamed("count(star_rating)", "customer_count")
customer_table.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16121565|             5|
|   48146680|             1|
|   11062912|             1|
|   51451778|             1|
|   27920838|             1|
+-----------+--------------+
only showing top 5 rows



In [14]:
mode="append"
jdbc_url = "jdbc:postgresql://:5432/my_data_class_db"
config = {"user":"root",
          "password": "",
          "driver":"org.postgresql.Driver"}

In [ ]:
customer_table.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
product_table.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)